In [ ]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [ ]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

### Goal: remove faulty documents
The goal is to remove faulty documents where mistakes were made when converting pdf into txt.

In [27]:
# load data
import ast
import pandas as pd

txtfiles = pd.read_csv(f"{cf.output_path}/txtfiles.csv")
# txtfiles['tokens'] = txtfiles['tokens'].apply(ast.literal_eval)

display(txtfiles)

,label,path,id,set,text,tokens,token_count,pdf_path,num_pages
0,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8336,test,BAeersteuning\n& |\nê u °\nGemeente Amsterdam ...,"['BAeersteuning', '&', '|', 'ê', 'u', '°', 'Ge...",9447,/home/azureuser/cloudfiles/code/blobfuse/raads...,18.0
1,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8337,train,ker Zj ze\nGemeente 4 AD\nAmsterdam ON\nhd P4 ...,"['ker', 'Zj', 'ze', 'Gemeente', '4', 'AD', 'Am...",20370,/home/azureuser/cloudfiles/code/blobfuse/raads...,74.0
2,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8338,train,G t\n% Amsterdam\n| (WD\n[Eenzaamheid pak je s...,"['G', 't', '%', 'Amsterdam', '|', '(', 'WD', '...",11004,/home/azureuser/cloudfiles/code/blobfuse/raads...,31.0
3,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8339,test,.\nps\nx ||\n“\nAe Tien Ja MK\nne 4 Je AJ & |\...,"['.', 'ps', 'x', '||', '“', 'Ae', 'Tien', 'Ja'...",12405,/home/azureuser/cloudfiles/code/blobfuse/raads...,34.0
4,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8340,val,Gemeente\n\nAmsterdam\nStaat van gezond gewich...,"['Gemeente', 'Amsterdam', 'Staat', 'van', 'gez...",11813,/home/azureuser/cloudfiles/code/blobfuse/raads...,48.0
...,...,...,...,...,...,...,...,...,...
33112,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32889,val,> Gemeente\nAmsterdam\nSchriftelijke vragen\nD...,"['>', 'Gemeente', 'Amsterdam', 'Schriftelijke'...",278,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0
33113,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32890,train,> Gemeente\nAmsterdam\n\nSchriftelijke vragen\...,"['>', 'Gemeente', 'Amsterdam', 'Schriftelijke'...",922,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
33114,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32891,train,> Gemeente\nAmsterdam\n\nSchriftelijke vragen\...,"['>', 'Gemeente', 'Amsterdam', 'Schriftelijke'...",339,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0
33115,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32892,train,> Gemeente\nAmsterdam\n\nSchriftelijke vragen\...,"['>', 'Gemeente', 'Amsterdam', 'Schriftelijke'...",633,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0


In [28]:
df = txtfiles.copy()

In [29]:
display(df['token_count'].describe())
class_describe = df.groupby('label')['token_count'].describe()
display(class_describe)

count     33117.000000
mean       2691.847933
std       13409.818780
min           3.000000
25%         291.000000
50%         533.000000
75%        1225.000000
max      275597.000000
Name: token_count, dtype: float64

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,696.773092,3462.293848,72.0,236.00,344.5,592.0,103700.0
Adviesaanvraag,2442.0,1644.871417,2993.302964,57.0,557.00,955.0,1672.0,77769.0
Agenda,3537.0,1048.904156,4244.783305,20.0,325.00,616.0,939.0,139648.0
Amendement,1969.0,2658.493652,7044.470410,30.0,236.00,359.0,1184.0,62304.0
Begroting,1967.0,13160.190646,46780.793626,40.0,248.00,385.0,2324.0,247184.0
Besluit,775.0,986.649032,1749.006500,70.0,168.50,439.0,1216.0,26523.0
Brief,1995.0,1764.259649,1728.612069,3.0,734.00,1269.0,2291.5,32957.0
Factsheets,234.0,6008.987179,14742.837685,112.0,1154.00,2753.5,5462.0,171297.0
Motie,8336.0,521.707893,1129.571368,105.0,234.00,292.5,402.0,36091.0
